In [4]:
# Save protein sequences as a FASTA file as required by the embedding generator
import pickle5 as pickle
from tqdm import tqdm
# Load pickle file
with open('data/datasetB_all_sequences.pkl', 'rb') as f:
    data = pickle.load(f)

# Write protein sequences in a fasta file
with open("data/datasetB_all_sequences.fasta", "w") as f:
    for key in tqdm(data):
        f.write('>'+key+'\n')
        f.write(data[key])
        f.write('\n')
f.close()

In [4]:

models = ['esm2_t33_650M_UR50D', 'esm2_t36_3B_UR50D', 'esm1_t34_670M_UR50S',
        'esm1_t6_43M_UR50S', 'esm2_t30_150M_UR50D', 'esm1v_t33_650M_UR90S_1',
        'esm1b_t33_650M_UR50S', 'esm2_t12_35M_UR50D', 'esm_msa1b_t12_100M_UR50S',
        'esm1_t34_670M_UR50D', 'esm_msa1_t12_100M_UR50S', 'esm2_t6_8M_UR50D',
        'esm_if1_gvp4_t16_142M_UR50']

import os, torch
# Process extracted emb
from Bio import SeqIO
fasta_sequences = SeqIO.parse(open('data/datasetB_all_sequences.fasta'),'fasta')

model_name='esm1_t6_43M_UR50S'

for fasta in fasta_sequences:
    #print('Filename:', datapoint)
    print(fasta.id, len(fasta.seq))
    emb = torch.load(model_name+'/'+fasta.id+'.pt')
    key = list(emb['representations'].keys())
    print(emb['representations'][key[0]].shape)
    print(emb['representations'][key[0]])
    break

1g58_A 196
torch.Size([196, 768])
tensor([[-0.7342,  0.7721, -0.2252,  ..., -0.7896,  1.2874,  0.9320],
        [-0.0884,  0.7979, -0.2805,  ...,  0.1627,  0.5936,  0.3086],
        [ 0.7058, -0.5040, -0.4714,  ..., -0.1014,  1.6905, -0.0378],
        ...,
        [ 0.0742, -0.1984,  0.2745,  ..., -0.9772,  1.1318, -0.0934],
        [-0.0326,  0.4182, -0.3380,  ..., -1.2327, -0.1898,  1.3206],
        [-0.5076, -0.3772,  0.8009,  ..., -1.1297,  0.0080, -0.9833]])


In [ ]:
from tqdm import tqdm
import torch, os
concatenated_dataset = []
model_name='esm1_t6_43M_UR50S'
#for fasta in tqdm(fasta_sequences):
i = 0
for datapoint in tqdm(os.listdir(model_name)):
    #emb = torch.load(model_name+'/'+fasta.id+'.pt')
    emb = torch.load(model_name+'/'+datapoint)['representations'][6]
    concatenated_dataset.append(emb)
    
final_dataset = torch.cat(concatenated_dataset)

In [9]:
torch.cat(concatenated_dataset).shape, emb.shape

(torch.Size([451563, 768]), torch.Size([48, 768]))

In [59]:
#import pandas as pd
from torch.utils.data import Dataset, DataLoader

class ProtEmbDataset(Dataset):
    """Protein emb dataset."""
    def __init__(self, model_name, transform=None):
        """
        Args:
            root_dir (string): Directory with all the datapoints.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.model_name = model_name
        self.filenames = os.listdir(model_name)

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        file = self.filenames[idx]
        emb = torch.load(model_name+'/'+file)
        key = list(emb['representations'].keys())
        sample = emb['representations'][key[0]]
        return sample

In [60]:
model_name = 'esm1_t6_43M_UR50S'
root_dir = 'data'
protein_dataset = ProtEmbDataset(model_name)

In [61]:
dataloader = DataLoader(protein_dataset, batch_size=1,
                        shuffle=True, num_workers=4)

In [62]:
for datapoint in dataloader:
    #datapoint = datapoint.view(-1)
    print(torch.squeeze(datapoint).shape)
    print(datapoint.shape)
    break

torch.Size([162, 768])
torch.Size([1, 162, 768])


In [69]:
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
import torch.utils.data as data
import numpy as np

# Create a validation and training set
samples_count = len(protein_dataset)
all_samples_indexes = list(range(samples_count))
np.random.shuffle(all_samples_indexes)

al_ratio = 0.2
val_end = int(samples_count * 0.2)
val_indexes = all_samples_indexes[0:val_end]
train_indexes = all_samples_indexes[val_end:]
assert len(val_indexes) + len(train_indexes) == samples_count , 'the split is not valid' 

sampler_train = data.SubsetRandomSampler(train_indexes)
sampler_val = data.SubsetRandomSampler(val_indexes)

dataloader_train = DataLoader(protein_dataset, batch_size=1, sampler = sampler_train, num_workers=4)
dataloader_val = DataLoader(protein_dataset, batch_size=1, sampler = sampler_val, num_workers=4)
# dataloader_test = DataLoader(protein_dataset, batch_size=1,
#                         shuffle=True, num_workers=4)

In [ ]:
class MultiTaskModel(nn.Module):
    """
    Creates a MTL model with the encoder from "arch" and with dropout multiplier ps.
    """
    def __init__(self, arch,ps=0.5):
        super(MultiTaskModel,self).__init__()
        self.encoder = create_body(arch)        #fastai function that creates an encoder given an architecture
        self.fc1 = create_head(1024,1,ps=ps)    #fastai function that creates a head
        self.fc2 = create_head(1024,2,ps=ps)
        self.fc3 = create_head(1024,5,ps=ps)

    def forward(self,x):

        x = self.encoder(x)
        age = torch.sigmoid(self.fc1(x))
        gender = self.fc2(x)
        ethnicity = self.fc3(x)

        return [ion1, ion2, ion3]